# Imports

In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import time
import pyttsx3
import speech_recognition as sr
import pytz
import random
import json
import pickle
import numpy as np
import nltk
import spacy
from spacy import displacy
from nltk.stem import WordNetLemmatizer
import re
from nltk import word_tokenize
import json
from tensorflow.keras.models import load_model
from nltk.tag.stanford import StanfordNERTagger
import os

In [2]:
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
MONTHS = ["january", "february", "march", "april", "may", "june","july", "august", "september","october", "november", "december"]
DAYS = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
DAY_EXTENTIONS = ["rd", "th", "st", "nd"]

# def speak(text):

In this function, we will pass text to it and have it be spoken through the computer.

In [3]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# def get_audio():

In this function, we will get what the user is saying from their mic and convert that into a string.

In [4]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""
        
        try:
            said = r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception: " + str(e))
            
    return said

First we need to set up the google calendar api: https://developers.google.com/calendar/quickstart/python?authuser=3
1) We will need to sign into a google account

2) Enable the google calendar api

3) Download the client configuration

4) Put the client configuration in our working directory

5) Install the google client library(in the website)

# def authenticate_google(name):

What this function essentially does is it store your credentials locally, so everytime we run our program we don't need to keep signing in.

In [5]:
def authenticate_google(name):
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(name + '.pickle'):
        with open(name + '.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(name + '.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)

    return service

# def get_events(day, service):

What this function does is it is able to return what events a user has for a particular day, by passing in two parameters which day we want to see the events for and the user data, so which user to we want to check the events for.

In [6]:
def get_events(day, service):
    
    time_list = []
    
    date = datetime.datetime.combine(day, datetime.datetime.min.time())
    end_date = datetime.datetime.combine(day, datetime.datetime.max.time())
    utc = pytz.UTC
    date = date.astimezone(utc)
    end_date = end_date.astimezone(utc)

    events_result = service.events().list(calendarId='primary', timeMin=date.isoformat(), timeMax=end_date.isoformat(),
                                        singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        speak('No upcoming events found.')
    else:
        speak(f"You have {len(events)} events on this day.")

        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])
            start_time = str(start.split("T")[1].split("-")[0])  # get the hour the event starts
            
            end = event['end'].get('dateTime', event['end'].get('date'))
            print(end, event['summary'])
            end_time = str(end.split("T")[1].split("-")[0])  # get the hour the event starts
            
            time_list.append((start_time, end_time))
            
            if int(start_time.split(":")[0]) < 12:  # if the event is in the morning
                start_time = start_time + "am"
            else:
                start_time = str(int(start_time.split(":")[0])-12)  # convert 24 hour time to regular
                start_time = start_time + "pm"  
                
            
            if int(end_time.split(":")[0]) < 12:  # if the event is in the morning
                end_time = end_time + "am"
            else:
                end_time = str(int(end_time.split(":")[0])-12)  # convert 24 hour time to regular
                end_time = end_time + "pm"  

            speak(event["summary"] + " at " + start_time + " to " + end_time)
    
    return time_list

# def get_date(text):

To figure out which date the user is talking about we first have to know what date it is today, for example if the user asks "What do I have on Sunday" and it is Friday meaning Sunday is two days from now we know that the user is asking about the Sunday 2 days from now. But if today is Sunday and the user is asking "What do I have on Sunday", then we assume that the user is probably asking about today so we return todays date. 

We also need to know which month it is, for example if the user is asking something about March, when the current month is December we can assume that they are talking about March in next year.

We will also check whether the user gave a specific date, for example if the user asks "What do I have on March 5th", the we need to understand that they asking for a day in the month "March" and the day they are asking for is the "5th" of March.

In [7]:
def get_date(text):
    text = text.lower()
    today = datetime.date.today()

    if text.count("today") > 0:
        return today

    day = -1
    day_of_week = -1
    month = -1
    year = today.year

    for word in text.split():
        if word in MONTHS:
            month = MONTHS.index(word) + 1
        elif word in DAYS:
            day_of_week = DAYS.index(word)
        elif word.isdigit():
            day = int(word)
        else:
            for ext in DAY_EXTENTIONS:
                found = word.find(ext)
                if found > 0:
                    try:
                        day = int(word[:found])
                    except:
                        pass

    if month < today.month and month != -1:  # if the month mentioned is before the current month set the year to the next
        year = year+1

    if month == -1 and day != -1:  # if we didn't find a month, but we have a day
        if day < today.day:
            month = today.month + 1
        else:
            month = today.month

    # if we only found a dta of the week
    if month == -1 and day == -1 and day_of_week != -1:
        current_day_of_week = today.weekday()
        dif = day_of_week - current_day_of_week

        if dif < 0:
            dif += 7
            if text.count("next") >= 1:
                dif += 7

        return today + datetime.timedelta(dif)

    if day != -1:
        return datetime.date(month=month, day=day, year=year)

# Log In:

Here we will asks our users to log in, we will prompt them asking how many users to add, for example if they wish to have two users we will ask them to authenticate two google accounts to allow us to get data from their google calendars.

As of now, this program only supports two users.

In [8]:
userNum = int(input("How many users: "))
users = []
id = 0

for i in range(userNum):
    service = authenticate_google("user_" + str(id))
    users.append(service)
    id = id + 1

How many users: 2
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598525128995-nhrijth6nrlk5ik0eahcrgmrogfbh3te.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50346%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=fZrRDY2vTAocyMk8Pyoq6snjEDwkDe&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598525128995-nhrijth6nrlk5ik0eahcrgmrogfbh3te.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50355%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=NNzLGcyExt7BaMnpvcgwfFqZUReplp&access_type=offline


In [9]:
# timesBounds = []
# for user in users:
#     bounds = []
#     speak("What are your morning bounds?")
#     mornBound = input("")
#     speak("What are your night bounds?")
#     nighBound = input("")
#     bounds.append(mornBound)
#     bounds.append(nighBound)
#     timesBounds.append(bounds)
# print(timesBounds)

# ChatBot

Here we will be using our model that we had built during training to build a chatbot, we will get input via voice from the user and based on what the model thinks the user is say we will reply adequately. What our model will do for us is it will label the statements the user is making and based on their statements the model will categorize it based on the tags we had implement in our intents json file. Based on what tag the model set the user input to be we can implement specific functionalities, such as, if the model determines that the user is trying to introduce themselves that would fall under the tag "name" and once we know that we can store the users name and greet them bye their name or address them by their name in later interactions. How we detect where the users name is in the statement they had inputed is by using something called spacy, spacy allows us to input a string to it and from that spacy is able to idetify what entities are in that string and if it identifies a person it will label that entity as a "PERSON" and we can find which word exactly is the users name and store that word. We can also use this when figuring out what day the user want to find their events for, if the user asks "What do I have on Sunday", "Sunday will be classified as a "DATE" and by knowing this we can pass in this information to our get_events() function and figure out which events each user has on that date.

In [10]:
nlp = spacy.load("en_core_web_sm")

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents-sch.json').read())

words = pickle.load(open('words.pkl', 'rb'))

classes = pickle.load(open('classes.pkl', 'rb'))

userName = ["user"]

model = load_model('chatbot.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json, text):
    endOfConvo = False
    doc = nlp(text)
    
    labels = []
    entities = []
    tokens = word_tokenize(message)
    for ent in doc.ents:
        labels.append(ent.label_)
        entities.append(ent.text)
    index_name = -1
    
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = " "
    for i in list_of_intents:
        if i['tag'] == tag:
            if (i['tag'] == 'greeting'):
                result = random.choice(i["responses"])
                break
            if (i['tag'] == 'goodbye'):
                endOfConvo = True
                result = (random.choice(i['responses']) % userName[0])
                break
            if (i['tag'] == 'name'):
                if "PERSON" in labels:
#                     print("Index: " + str(labels.index("PERSON")))
                    index_name = labels.index("PERSON")
                userName[0] = str(entities[index_name])
                result = (random.choice(i['responses']) % userName[0])
                break
            if (i['tag'] == 'schedule'):
                times = []
                getValues = []
                if "DATE" in labels:
#                     print("Index: " + str(labels.index("DATE")))
                    index_name = labels.index("DATE")
                day = str(entities[index_name])
                date = get_date(text.lower())
                if date:
                    times.append(get_events(date,users[0]))
                    times.append(get_events(date,users[1]))
#                     find_time(timesBounds, times)
#                     print(times)
                else:
                    speak("what do you mean by that?")
                result = ""
                break
    return result, endOfConvo

speak("Hello there!")
while True:
    text = get_audio()
    text = re.sub(r'[^\w\s]', '', text)
    textReg = text
    
    text = text.lower()
    
    message = text
    
    doc = nlp(text)
    
    ints = predict_class(text)
    res, endOfConvo = get_response(ints, intents, textReg)
    speak(str(res))
    print(str(res))
    if endOfConvo: 
        break

hello
Hello, how can I help?
my name is Bobby
Pleasure to meet you, Bobby
what do I have on Saturday
2021-04-10T14:00:00-05:00 Tech Team Meeting
2021-04-10T14:30:00-05:00 Tech Team Meeting

talk to you later
Talk to you soon, Bobby!


In [187]:
# def find_time(bounds, times):
#     freeHrs = []
#     for i in times:
#         userTime = []
#         for j in i:
#             hrs = []
#             for a in j:
#                 print(a.split(":"))
#                 hrs.append(a.split(":"))
#             userTime.append(hrs)
#         freeHrs.append(userTime)
#     print(freeHrs)
#     mins = []
#     inc = 0
#     for i in range(len(freeHrs)):
#         for j in range(i):
#             for a in range(j):
#                 freeHrs[i][j][a] = int(freeHrs[i][j][a])
#                 min = freeHrs[i][j][a][0] * 60
#                 min = min + freeHrs[i][j][a][0]
#                 print("min: " + min)
#                 mins.append(min)
#                 inc = inc + 1
#     print(mins)
    